### ⏬ Data quality


In [ ]:
#loading dataset
from module_list import *

In [ ]:
url = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/tumors.csv"
df=pd.read_csv(url,header=0)
df=df.sample(frac=1).reset_index(drop=True)
df.head(2)
# Define X and y  
X = df[["leukocyte_count", "blood_pressure"]].values
y = df["tumor_class"].values
print ("X: ", np.shape(X))
print ("y: ", np.shape(y))

In [ ]:
classes=np.unique(y)
import collections
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
# bb=LabelEncoder.fit(y_train)
# encoder=OrdinalEncoder()
label_encoder=LabelEncoder().fit(y.reshape(-1,1))
# encoder.fit_transform(y.reshape(-1,1))

list(label_encoder.classes_),list(collections.Counter(y))


In [ ]:
pd.plotting.scatter_matrix(df, figsize=(5, 5));
df.corr()

In [ ]:
url = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/tumors.csv"
df=pd.read_csv(url,header=0)
df=df.sample(frac=1).reset_index(drop=True)
df.head(2)
# Define X and y  
X = df[["leukocyte_count", "blood_pressure"]].values
y = df["tumor_class"].values
print ("X: ", np.shape(X))
print ("y: ", np.shape(y))

main=Main(X,y)
main.SLP()
main.ploting(X=X[:,0],y=X[:,1])
# main.status()
#modeling
model_2=Lin2(2,10)

#training
training(X_train,X_test,y_train,y_test,model_2)
plt.scatter(df['leukocyte_count'].mean(),np.mean(df.blood_pressure), s=200,
            c="b", edgecolor="w", linewidth=2)
plt.scatter(X[:,0],X[:,1],c='red',alpha=0.2)
ax = plt.axes()
ax.set_facecolor('#1CC4AF')
# Annotate
plt.annotate("true: malignant,\npred: malignant",
        color="black",
        xy=(df['leukocyte_count'].mean(),np.mean(df.blood_pressure)),
        xytext=(0.4, 0.75),
        textcoords="figure fraction",
        fontsize=12,
        arrowprops=dict(facecolor="white", shrink=0.01))



In [ ]:
import random
from module_list import *
seed=1234
def set_seed(seed):
  np.random.seed(seed);
  random.seed(seed);
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed);
  torch.cuda.manual_seed_all(seed);
set_seed(seed)
#set device
cuda=True
device=torch.device('cuda' if(torch.cuda.is_available() and cuda)else 'cpu')
torch.set_default_tensor_type('torch.FloatTensor')
if device.type =='cuda':
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
print(device)
url = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/spiral.csv"
df=pd.read_csv(url,header=0)
df=df.sample(frac=1).reset_index(drop=True)
X=df[['X1','X2']].values
y=df['color'].values


In [ ]:
main=Main(X,y)
main.SLP()
main.ploting(X=X[:,0],
             y=X[:,1])

📢
###DATALOADER

In [ ]:
#create dataloaders
set_seed(seed)
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
      return len(self.y)

  def __str__(self):
        return f"<Dataset(N={len(self)})>"

  def __getitem__(self, index):
    X = self.X[index]
    y = self.y[index]
    return [X, y]

  def collate_fn(self, batch):
        """Processing on a batch."""
        # Get inputs
        batch = np.array(batch)
        X = np.stack(batch[:, 0], axis=0)
        y = batch[:, 1]
        # Cast
        X = torch.FloatTensor(X.astype(np.float32))
        y = torch.LongTensor(y.astype(np.int32))
        return X, y

  def create_dataloader(self, batch_size, shuffle=False, drop_last=False):
    return torch.utils.data.DataLoader(
        dataset=self, batch_size=batch_size, collate_fn=self.collate_fn,
        shuffle=shuffle, drop_last=drop_last, pin_memory=True)

batch_size=64
training_data=Dataset(X=X_train.cpu(),y=y_train.cpu())
test_data=Dataset(X=X_test,y=y_test)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
train_dataloader2=training_data.create_dataloader(batch_size=batch_size)
test_dataloader2=test_data.create_dataloader(batch_size=batch_size)
# batch_X,batch_y=next(iter(train_dataloader ))
# batch_X_test,batch_y_test=next(iter(test_dataloader))
# train_stackedloader=np.stack([batch_X[:,0],batch_X[:,1],batch_y]).T
# test_stackedloader=np.stack([batch_X_test[:,0],batch_X_test[:,1],batch_y]).T

len(train_dataloader), len(test_dataloader),len(train_dataloader2),len(test_dataloader2)


⏰
###__MODEL__

In [ ]:

class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(2,100),
      nn.ReLU(),
      # nn.Dropout(p=0.2),
      nn.Linear(100,100),

      # nn.ReLU(),
      nn.Linear(100, 3),
    )
  def forward(self, x):
    '''Forward pass'''
    x=self.layers(x)
    return x

# model_2=MLP(hidden_dim=50,dropout_p=0.2)

model_2=MLP()


❎
###TRAINING AND TEST

In [ ]:
# Define Loss
loss_fn = nn.CrossEntropyLoss()

# Define optimizer & scheduler
optimizer = Adam(model_2.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=3)

class Trainer(object):
  def __init__(self, model, device, loss_fn, optimizer,EPOCHS,scheduler=None):

    super(Trainer, self).__init__()
    # Set params
    self.model = model
    self.device = device
    self.loss_fn = loss_fn
    self.optimizer = optimizer
    self.scheduler = scheduler
    self.EPOCHS=EPOCHS

  def train_step(self,dataloader):

    """Train step."""
    # Set model to train mode
    size=len(dataloader.dataset) # Size of Training Data
    num_batches = len(dataloader)

    self.model.train()
    loss,correct=0,0
    # Iterate over train batches
    for i,(X,y) in enumerate(dataloader):        
      X,y=X.to(device),y.to(device)

      z=self.model(X.float())
      # print(z.shape,y.shape)
      # break
      FORD = self.loss_fn(z, y.squeeze())
      loss += FORD.detach().item()

      self.optimizer.zero_grad()
      FORD.backward()
      self.optimizer.step()
      correct+=(z.argmax(1)==y).type(torch.float).sum().item()

    loss /= num_batches
    correct /= size # Correct is checked with every data point in data but loss is calculated wiht every per Batch.
    return correct
    #     batch = [item.to(self.device) for item in batch]  # Set device
    #     inputs, targets = batch[:-1], batch[-1]
    #     self.optimizer.zero_grad()  # Reset gradients
    #     FORD = self.model(inputs)  # Forward pass
    #     J_loss = self.loss_fn(FORD, targets)  # Define loss
    #     J_loss.backward()  # Backward pass
    #     self.optimizer.step()  # Update weights

    #     # Cumulative Metrics
    #     loss += (J_loss.detach().item() - loss) / (i + 1)
    # return loss

  def eval_step(self, dataloader):
    """Validation or test step."""
    # Set model to eval mode
    self.model.eval()
    size=len(dataloader.dataset) # Size of Training Data
    num_batches = len(dataloader)
    correct,loss = 0.0, 0.0
    # y_trues, y_probs = [], []

    # Iterate over val batches
    for i in range(self.EPOCHS):
      with torch.inference_mode():
        for batch,(X,y) in enumerate(dataloader):
          X,y=X.to(device),y.to(device)
          # Step
          z=self.model(X.float())
          loss +=self.loss_fn(z,y.squeeze()).item()
          # correct+=(z.argmax(1)==y).type(torch.float).sum().item()
          # correct=torch.eq(z)
          correct+=torch.eq(z.argmax(1),y.reshape(1,-1)).sum().item()
        
        
        
      
      correct/= size
      loss/= num_batches
      if i%30==0:
        print(f'Epochs {i} | Evaulate_loss {loss:.2f} | Accuracy: {correct*100:2f} ')
    return loss

  def Train(self,train_dataloader, test_dataloader):
    correct=self.train_step(dataloader=train_dataloader)
    FORD=self.eval_step(dataloader=test_dataloader)
      
trainer=Trainer(model=model_2,device=device,loss_fn=loss_fn,
                optimizer=optimizer,scheduler=None,EPOCHS=200)

# trainer.train_step(dataloader=train_dataloader)
trainer.Train(train_dataloader=train_dataloader,test_dataloader=test_dataloader)

In [ ]:
import json
from sklearn.metrics import precision_recall_fscore_support
def get_metrics(y_pred, y_true, classes):
    """Per-class performance metrics."""
    # Performance
    performance = {"overall": {}, "class": {}}

    metrics = precision_recall_fscore_support(y_pred,y_true.reshape(-1,1))
    performance["overall"]["precision"] = metrics[0][1]
    performance["overall"]["recall"] = metrics[1][1]
    performance["overall"]["f1"] = metrics[2][1]
    performance["overall"]["num_samples"] = np.float64(len(y_true))

    # Per-class performance
    
    return performance


test_loss, y_true, y_prob = trainer.eval_step(dataloader=test_dataloader)


y_pred = np.argmax(y_prob, axis=1)
y_pred.shape
# performance = get_metrics(y_true=y_test, y_pred=y_pred, classes=3)

# print (json.dumps(performance["overall"], indent=2))


🔒
**`OLD CLASS`**
1. **MAIN class for preprocessing Data**
2. **MODEL Class**
3. **Testing *italicized text* and Training**

In [ ]:
from module_list import *
class Main(object):
  """
    Preprocessing Data in SLP;
     Modeling in Lin2;
     Training in Training
  """
  def __init__(self,X,Y):    
    super().__init__()
    self.X=X
    self.Y=Y
    self.condition = True # " True" Or "False"
      
    # if self.condition:
        # self.torch = import_module('torch') #import torch inside the main Class
      
  def SLP(self):
    global X_train,X_test,y_train,y_test,y_1
    # Split the Dataset for Traning and Testing
    y_1=self.Y
    X_train,X_test,y_train,y_test=train_test_split(self.X,self.Y, train_size=0.8)
    
    # Standardization on X_trian,X_test
    scaler=StandardScaler()
    encoder=OrdinalEncoder()
    
    # Also can written in this form : 
    """
      ->Becareful with the shape of input for standardizaiton.0D is not allowed.
        it will Raise an Error for shape of the data. solution: y.reshape(-1,1)
        or if our Data is tensor use this: torch.reshape(y, (-1,1) ) 

        scaler=StandardScaler.fit(y)
        X_train,X_test=scaler.transform(y)
    """
    try:
      X_train=scaler.fit_transform(X_train)
      X_test=scaler.fit_transform(X_test)
      X_train=torch.tensor(X_train)
      X_test=torch.tensor(X_test)    
    except: 
      print(f'Data didn"t convert to Tensor.\n'
            f'Something wrong with the shape.Please reshape and then Try....')

    # Decoting the categorical columns on y
    if isinstance(self.Y[1], str):
      y_train=encoder.fit_transform(y_train.reshape(-1,1))
      y_test=encoder.fit_transform(y_test.reshape(-1,1))
      y_train=torch.LongTensor(y_train)
      y_test=torch.LongTensor(y_test)

    
    # If target has more than 
    if len(self.X.shape)>1:
      
      # Type of problem
      if  isinstance(self.Y[1], str) | self.X.shape[1]> 1 :
        print('-> Multinomial Logistic Regression <-')

      # If data is less than one column and are not Categorical we define it as Linear and standard the y_columns for getting more Efficient result on testing
      elif  isinstance(self.Y[1], str)| self.X.shape[1]<1:
        print('->Binomial Logistic Regression <-')
      else: 
        print('-> Linear Regression Problem <-')
        y_train=scaler.fit_transform(y_train)
        y_train=torch.tensor(y_train)
        y_test=scaler.fit_transform(y_test)
        y_test=torch.tensor(y_test)
    
    # return (X_train,X_test,y_train,y_test)
    
  def ploting(self,**kwargs): 
    # Print('ploting selected data')
    
    colors={}
    
    class_num=len(np.unique(self.Y))
    # Generate color for each class in Dat
    colors={}
    for i in np.unique(self.Y):
      rand1=tuple(np.random.random(size=3))
      colors.update({f'{i}':rand1})

    plt.scatter(self.X[:, 0],
                self.X[:, 1], c=[colors[_y] for _y in y_1],
            s=25, edgecolors="k")

    # plt.title(kwargs.get('title'))
    # plt.xlabel(kwargs.get('xlabel'))
    # plt.legend(kwargs.get('legend'))
    plt.show()

  #Checking for more information about data type and type of our Data
  def status(self):
    print(f'X_train: {type(X_train)} --> {X_train.dtype} \n'
          f'X_test: {type(X_test)} --> {X_test.dtype} \n \n'
          f'y_train: {type(y_train)} --> {y_train.dtype} \n'
          f'y_test: {type(y_test)} --> {y_test.dtype}'    )
    

class Lin2(nn.Module):  
  def __init__(self,input_dim,output_dim):
      super().__init__()
      """hi"""
      self.input_dim=input_dim
      self.output_dim=output_dim

      # Number of the classes
      classes_num=len(np.unique(y_train))

      self.layers = nn.Sequential(
      nn.Linear(self.input_dim, self.output_dim),
      nn.Tanh(), # <- add activation function
      nn.Linear(self.output_dim, self.output_dim),
      # nn.Tanh(),
      nn.Linear(self.output_dim, classes_num),
      # nn.ReLU()
      )    
     
      # Also we can use another form of using Linear Layers
      """
      self.lin1 = nn.Linear(self.input_dim,self.output_dim)
      self.lin2 = nn.Linear(self.output_dim,classes_num)
      """

  #Loss Function for categorical problem
  loss_fn = nn.CrossEntropyLoss()

  # Forwarding the X_train to the Neural Network 
  def forward(self, x):
      return self.layers(x)

  # For categorical problem, we sum all the True label with pred label to validate how much of are the labels that predicted were True?
  def accuracy_fn(self,**kwargs):
    n_correct = torch.eq(kwargs.get('pred'), kwargs.get('y_true')).sum().item()
    accuracy = (n_correct / len(kwargs.get('pred'))) * 100
    return accuracy


# Training
def training(X_train,X_test,y_train,y_test,model):
  """"X_train,X_test,y_train,y_test,model"""
  optimizer = Adam(model.parameters(), lr=0.2)

  for epoch in range(100):
    # Forward pass
    y_pred = model(X_train.float())
    
    # Loss function
    loss = model.loss_fn(y_pred, y_train.squeeze())
    
    
    # Zero all gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    if epoch%20==0:
      predictions = y_pred.max(dim=1)[1] # class
      # print(predictions.shape,y_train.squeeze().shape)
      """
      All the tensor must has the same Dim.If Dim aren't the same the 
      accuracy we would be have a problem.Use reshape method instead.
      """
      
      accuracy = model.accuracy_fn(pred=predictions,y_true=y_train.squeeze())
      
      print (f"Epoch: {epoch} | loss: {loss:.2f}, accuracy: {accuracy:.1f}")


def plot_multiclass_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    cmap = plt.cm.Spectral
    
    X_test = torch.from_numpy(np.c_[xx.ravel(), yy.ravel()]).float()
    y_pred = F.softmax(model(X_test), dim=1) 
    _, y_pred = y_pred.max(dim=1)
    y_pred = y_pred.reshape(xx.shape)
    plt.figure(figsize=(10,5))
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Train")
    plt.xlabel('X')
    plt.ylabel('Y')
    

In [ ]:
"""Get the class of categorical columns
   1. label_encoder=Labelencoder().fit(y)
      list(label_encoder.classses_)
  2. list(colleciton.Counter(y))
   """

   #example of labelencoding; -> two ways
c={'bird','animal','sea','cat'}
"""  set are not Ordered   """

print(
    list((LabelEncoder().fit(list(c))).classes_)
    )
print(
  list(collections.Counter(list(c) ))  
  )


""" We CANT global var of a used var, Global var must first empty"""
""" Reset all variable---> %reset or %reset -f"""